# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [17]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Activation

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [45]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1759622788318959667
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 312213504
locality {
  bus_id: 1
}
incarnation: 2143228179873907666
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    # I observe that '?.,' punctuations exist both in the eng and fr dataset. 
    # Here I removed '?.,' from the filters therefore my model will take those punctuations as a word and will try to place them while making a translation. 
    # I am aware that this will bring additional complexity to the model but I want to test if my model can handle that and will try to keep the error rates within acceptable limits.
    my_tokenizer = Tokenizer(filters='!"#$%&()*+-/:;<=>@[\]^_`{|}~')
    my_tokenizer.fit_on_texts(x)
    x_token = my_tokenizer.texts_to_sequences(x)

    return x_token, my_tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'.': 1, 'the': 2, 'quick': 3, 'a': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'lazy': 9, 'dog': 10, 'by': 11, 'jove': 12, ',': 13, 'my': 14, 'study': 15, 'of': 16, 'lexicography': 17, 'won': 18, 'prize': 19, 'this': 20, 'is': 21, 'short': 22, 'sentence': 23}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 5, 6, 7, 8, 2, 9, 10, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [11, 12, 13, 14, 3, 15, 16, 17, 18, 4, 19, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [20, 21, 4, 22, 23, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    my_padding = pad_sequences(x, maxlen=length,padding='post')
    return my_padding

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  5  6  7  8  2  9 10  1]
  Output: [ 2  3  5  6  7  8  2  9 10  1  0  0]
Sequence 2 in x
  Input:  [11 12 13 14  3 15 16 17 18  4 19  1]
  Output: [11 12 13 14  3 15 16 17 18  4 19  1]
Sequence 3 in x
  Input:  [20 21  4 22 23  1]
  Output: [20 21  4 22 23  1  0  0  0  0  0  0]


# Extra Work : Punctuations and Further Data Cleaning

# Punctuations

I observe that '?.,' punctuations exist both in the english and french dataset. 
I intentionally leave them in the input and target data and considered each punctuation as a word. I am aware that this will  bring additional complexity to the model but I strongly curious if my model can handle punctuations while translating.


# Further Data Cleaning
Before running the automated preprocess I investigated both data and found the following improvement areas in order to get cleaner data for modelling.  With these corrections unique number of words decreased from 227 to 202 in English, 355 to 331 in French without loosing any information in the data. I expect these corrections to improve my modelling accuracy. 

    Observations to be corrected in French Data:
    1. "États", "états" and "etats" exist seperately(mistype and lower case mistake). I made them both "états".
    2. à and a both exist as words.(mistype) They are the same word. We replaced them all to à
    3. "aimé." vs "aimé" and "préféré." vs "préféré" exist as distinct words.(space delimitation mistake) replaced them with "aimé ." and "préféré ." 
    4. Both "rouillé" and "rouille" exist as distinct words.(mistype) Replace them with "rouillé"
    5. Both "californie" and "california" exist.(mistype) We replace them both with californie
    6. My limited french knowledge tells me that it is wise to define "l'", "n'", and "j'" as seperate words. Because all these are seperate words and appear as "l'animal" only if the following word start with a vowel. I replaced "l'animal" with "l' animal" so that  "l'animal" and "animal" are not distinct words in the data. Done the same thing for all l', n' and j' occurences

    Observations to be corrected in English Data:
    1. there is space delimitation mistakes before periods '.' widely. Corrected them. 

I implemented these corrections below as eng_corrections() and fr_corrections() functions and inserted these functions to the preprocess pipeline.

In [9]:
def eng_corrections(x):
    
    x = [i.replace('.', ' .') for i in x]
    x = [i.replace('  .', ' .') for i in x] #Remove double spaces
    
    return x

def fr_corrections(y):
    #correction 1
    y= [j.replace('É', 'é') for j in y]
    y= [j.replace('etats', 'états') for j in y]
    
    #correction 2
    y= [j.replace(' a ', ' à ') for j in y]
   
    #correction 3
    y= [j.replace('.', ' .') for j in y]
    y= [j.replace('  .', ' .') for j in y] #Remove double spaces
    
    #correction 4
    y= [j.replace('rouille', 'rouillé') for j in y]
    
    #correction 5
    y= [j.replace('california', 'californie') for j in y]
    
    #correction 6
    y= [j.replace('l\'', 'l\' ') for j in y]
    y= [j.replace('n\'', 'n\' ') for j in y]
    y= [j.replace('j\'', 'j\' ') for j in y]

    y= [j.replace('l\'  ', 'l\' ') for j in y] #Remove double spaces
    y= [j.replace('n\'  ', 'n\' ') for j in y]
    y= [j.replace('j\'  ', 'j\' ') for j in y]
    
    return y

### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x = eng_corrections(x)
    preprocess_y = fr_corrections(y)
    
    preprocess_x, x_tk = tokenize(preprocess_x)
    preprocess_y, y_tk = tokenize(preprocess_y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 17
Max French sentence length: 23
English vocabulary size: 202
French vocabulary size: 331


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [52]:

from keras.models import Sequential
from keras.layers import Activation

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers

    learning_rate = 0.01
    model = Sequential()
    model.add(GRU(output_sequence_length, input_shape = input_shape[1:], return_sequences=True))
    model.add(Dense(500, activation="relu"))
    model.add(Dense(french_vocab_size, activation="softmax"))
  
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
#plus one for the padding character
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))



Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 101us/step - loss: 2.2348 - acc: 0.4818 - val_loss: 1.6366 - val_acc: 0.5610
Epoch 2/10
110288/110288 [==============================] - 9s 84us/step - loss: 1.4800 - acc: 0.5842 - val_loss: 1.3900 - val_acc: 0.6002
Epoch 3/10
110288/110288 [==============================] - 9s 84us/step - loss: 1.3379 - acc: 0.6055 - val_loss: 1.2854 - val_acc: 0.6180
Epoch 4/10
110288/110288 [==============================] - 9s 84us/step - loss: 1.2554 - acc: 0.6202 - val_loss: 1.2232 - val_acc: 0.6312
Epoch 5/10
110288/110288 [==============================] - 9s 85us/step - loss: 1.1954 - acc: 0.6333 - val_loss: 1.1703 - val_acc: 0.6395
Epoch 6/10
110288/110288 [==============================] - 9s 84us/step - loss: 1.1498 - acc: 0.6447 - val_loss: 1.1450 - val_acc: 0.6460
Epoch 7/10
110288/110288 [==============================] - 9s 85us/step - loss: 1.1102 - acc: 0.6533 - val_loss:

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [21]:

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.01
    model = Sequential()
    model.add(Embedding(english_vocab_size, 64, input_length=output_sequence_length,mask_zero = False))
    model.add(GRU(output_sequence_length, return_sequences=True))
    model.add(Dense(500, activation="relu"))
    model.add(Dense(french_vocab_size, activation="softmax"))
  
    
    model.compile(loss=sparse_categorical_crossentropy,
            optimizer=Adam(learning_rate),
            metrics=['accuracy'])
    return model


tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
#plus one for the padding character
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))



Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 103us/step - loss: 1.5153 - acc: 0.6257 - val_loss: 0.6925 - val_acc: 0.7717
Epoch 2/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.5517 - acc: 0.8232 - val_loss: 0.4731 - val_acc: 0.8453
Epoch 3/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.4363 - acc: 0.8574 - val_loss: 0.4132 - val_acc: 0.8624
Epoch 4/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.3795 - acc: 0.8753 - val_loss: 0.3562 - val_acc: 0.8838
Epoch 5/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.3462 - acc: 0.8860 - val_loss: 0.3312 - val_acc: 0.8922
Epoch 6/10
110288/110288 [==============================] - 10s 93us/step - loss: 0.3215 - acc: 0.8944 - val_loss: 0.3180 - val_acc: 0.8944
Epoch 7/10
110288/110288 [==============================] - 10s 93us/step - loss: 0.3058 - acc: 0.8992 - val

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [23]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.01
    model = Sequential()
    model.add(Bidirectional(GRU(output_sequence_length, return_sequences=True),input_shape = input_shape[1:]))
    model.add(Dense(500, activation="relu"))
    model.add(Dense(french_vocab_size, activation="softmax"))
  
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)



# TODO: Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
#plus one for the padding character
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))



Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 14s 125us/step - loss: 1.9393 - acc: 0.5388 - val_loss: 1.3679 - val_acc: 0.6085
Epoch 2/10
110288/110288 [==============================] - 13s 114us/step - loss: 1.2744 - acc: 0.6218 - val_loss: 1.2011 - val_acc: 0.6334
Epoch 3/10
110288/110288 [==============================] - 13s 114us/step - loss: 1.1649 - acc: 0.6402 - val_loss: 1.1299 - val_acc: 0.6470
Epoch 4/10
110288/110288 [==============================] - 13s 113us/step - loss: 1.1010 - acc: 0.6518 - val_loss: 1.0768 - val_acc: 0.6538
Epoch 5/10
110288/110288 [==============================] - 13s 114us/step - loss: 1.0529 - acc: 0.6619 - val_loss: 1.0399 - val_acc: 0.6647
Epoch 6/10
110288/110288 [==============================] - 13s 113us/step - loss: 1.0103 - acc: 0.6719 - val_loss: 0.9888 - val_acc: 0.6745
Epoch 7/10
110288/110288 [==============================] - 12s 113us/step - loss: 0.9800 - acc: 0.6767

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    return None
tests.test_encdec_model(encdec_model)

#I Skip that since my Deadline is really soon:(

# OPTIONAL: Train and Print prediction(s)

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [41]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    learning_rate = 0.01
    model = Sequential()
    model.add(Embedding(english_vocab_size, 64, input_length=output_sequence_length,mask_zero = False))
    model.add(Bidirectional(GRU(output_sequence_length, return_sequences=True)))
    model.add(Dense(500, activation="relu"))
    model.add(Dense(french_vocab_size, activation="softmax"))
  
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

#tests.test_model_final(model_final)
#there is a problem with the test procedure, it may be a bug, I comment it out. 


print('Final Model Loaded')
# TODO: Train the final model

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [44]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    tmp_x = pad(x, y.shape[1])

# Train the neural network
    
    model = model_final(
        tmp_x.shape,
        y.shape[1],
        len(x_tk.word_index)+1,
        len(y_tk.word_index)+1)#plus one for the padding character
    
    model.fit(tmp_x, y, batch_size=1024, epochs=15, validation_split=0.2)

    
    #I needed to make a modifications below.
    #modified the padding length to target length. replaced (maxlen=x.shape[-1]) with (maxlen = y.shape[1])
    #and replaced "np.array([sentence[0], x[0]])" with "np.array([sentence[0], tmp_x[0]])"
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=y.shape[1], padding='post')
    sentences = np.array([sentence[0], tmp_x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/15
110288/110288 [==============================] - 16s 148us/step - loss: 1.4489 - acc: 0.6483 - val_loss: 0.5951 - val_acc: 0.8107
Epoch 2/15
110288/110288 [==============================] - 14s 129us/step - loss: 0.4545 - acc: 0.8561 - val_loss: 0.3643 - val_acc: 0.8831
Epoch 3/15
110288/110288 [==============================] - 14s 129us/step - loss: 0.3259 - acc: 0.8955 - val_loss: 0.2928 - val_acc: 0.9059
Epoch 4/15
110288/110288 [==============================] - 14s 130us/step - loss: 0.2697 - acc: 0.9133 - val_loss: 0.2569 - val_acc: 0.9173
Epoch 5/15
110288/110288 [==============================] - 14s 129us/step - loss: 0.2351 - acc: 0.9254 - val_loss: 0.2283 - val_acc: 0.9273
Epoch 6/15
110288/110288 [==============================] - 14s 129us/step - loss: 0.2121 - acc: 0.9327 - val_loss: 0.2042 - val_acc: 0.9355
Epoch 7/15
110288/110288 [==============================] - 14s 129us/step - loss: 0.1919 - acc: 0.9392

# FINAL COMMENTS

1. I included puctuations in the model as an extra work. I see that my model can handle punctuations as I hoped for. That was good news for me. 
2. Again as an extra modification, I seperated the (l' , j' , n') as distinct words. I see that my model can handle "l'" as seperate words as in "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ." prediction. That was something else I hoped to work, it was nice to see that. 
3. In the embedding model, I tried mask_zero = True in the embedding layer and hoped that the PAD word will be considered as a distinct dimension in the embedding and this will help the prediction accuract. But it did not worked as I hoped, it removed PADs from the predictions where predictions repeated the last word of the sentence as some kind of a padding strategy. This did not looked nice of course, I removed that parameter. 
4. The greatest question in my mind: as instructed in the notebook I gave the model sentences as sequences of integers where each integer refers to a word. But this looks like time series prediction and treats words as ordinal items which is not truein real life. Shouldn't we convert them to one-hot-encodings? Or does Keras handle that within modelling? I would be really happy if the reviewer could enlighten me on that:) 

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [2]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?